In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

In [3]:
from argparse import Namespace

args = Namespace(
    random_state=42,
    preprocessing=True,
    test_size=0.2,
    num_queries=100,
    batch_size = 5,
    input_path="./input/feedback-prize-2021/"
)

In [24]:
from datasets import build_fdb_data

dataset, val, evaluator = build_fdb_data(args)

100%|██████████| 15594/15594 [00:02<00:00, 6833.87it/s]


In [36]:
from torch.utils.data import DataLoader

def collate_fn_2(batch):
    docs = [data[0] for data in batch]
    infos = [data[2] for data in batch]
    targets = [data[1] for data in batch]
    return docs, targets, infos

def collate_fn_3()

dl = DataLoader(dataset, args.batch_size, collate_fn=collate_fn_2)

batch = next(iter(dl))

In [37]:
docs, targets, infos = batch

In [38]:
targets

[{'labels': tensor([5, 3, 0, 3, 0, 3, 0, 3, 1]),
  'boxes': tensor([[0.0245, 0.0508],
          [0.1215, 0.1431],
          [0.2392, 0.0885],
          [0.4859, 0.4049],
          [0.7119, 0.0471],
          [0.7599, 0.0452],
          [0.7891, 0.0094],
          [0.8192, 0.0471],
          [0.9266, 0.1563]])},
 {'labels': tensor([4, 2, 6, 0, 0, 0, 5, 0, 3, 0, 3, 0, 3, 1]),
  'boxes': tensor([[0.0479, 0.0973],
          [0.1108, 0.0284],
          [0.1347, 0.0195],
          [0.1594, 0.0299],
          [0.1879, 0.0210],
          [0.2066, 0.0075],
          [0.2290, 0.0195],
          [0.2478, 0.0180],
          [0.4042, 0.2949],
          [0.5689, 0.0344],
          [0.6774, 0.1826],
          [0.7852, 0.0329],
          [0.8585, 0.1138],
          [0.9581, 0.0853]])},
 {'labels': tensor([5, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 3, 1]),
  'boxes': tensor([[0.0107, 0.0225],
          [0.0270, 0.0101],
          [0.0417, 0.0191],
          [0.0721, 0.0417],
          [0.1025, 0.0191],
      

In [39]:
num_classes = 7
predictions = {
    'pred_logits': torch.rand([args.batch_size, args.num_queries, num_classes + 1]) * torch.Tensor([[1] * num_classes + [10]]),
    'pred_boxes': torch.rand([args.batch_size, args.num_queries, 2]) * torch.Tensor([[1, 0.1]])
}

In [40]:
from models.matcher import HungarianMatcher
from models.criterion import CriterionDETR

matcher = HungarianMatcher()

weight_dict = {'loss_ce': 1., 'loss_bbox': 1., 'loss_giou': 1.}
losses = ['labels', 'boxes', 'cardinality']

criterion = CriterionDETR(num_classes, matcher=matcher, weight_dict=weight_dict,
                            eos_coef=0.1, losses=losses)

In [41]:
criterion(predictions, targets)

{'loss_ce': tensor(2.4909),
 'loss_bbox': tensor(0.0802),
 'loss_giou': tensor(0.4866),
 'cardinality_error': tensor(6.2000)}

In [42]:
batch[2]

[{'id': '97E4E42863A3', 'length': 531},
 {'id': '03BA5E41C5E4', 'length': 668},
 {'id': 'EB793B72A8C4', 'length': 888},
 {'id': '0119F710D008', 'length': 203},
 {'id': 'F00B4D036D97', 'length': 209}]